One Day SSAM
--

1. Data from GeoNet FDSN web server.
2. Data decimated to 50 sps, as don't ned frequencies above 25 Hz.
3. Data windows do not overlap. Window length 3 minutes.
4. Frequency resolution 0.1 Hz. FFT reampled to achieve this.
5. Sensitivity removed from raw data.
6. Colour map bounded by 10 and 5e6.

In [ ]:
import numpy as np
from numpy.fft import rfft
from obspy.core import read, Trace, Stream, UTCDateTime
from obspy.signal.invsim import cosine_taper
from obspy.clients.fdsn import Client
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline

In [ ]:
#data window (seconds), should be a finite fraction of an hour
win = 180
#frequency interval (hertz)
fint = 0.1

In [ ]:
#fdsn client
#client = Client("http://beta-service.geonet.org.nz")
client = Client("GEONET")

In [ ]:
def daywave(date, net, sta, loc, cmp):
  #get a day of data from a FDSN web server
  global st
  time = UTCDateTime(date)
  st = client.get_waveforms(net, sta, loc, cmp, time, time + 86400, attach_response=True)

In [ ]:
date = '2012-08-06'
net = 'NZ'
sta = 'KRVZ'
cmp = 'EHZ'
loc = '10'

In [ ]:
daywave(date, net, sta, loc, cmp)
st.remove_sensitivity()

st.merge(fill_value = 'interpolate')
tr = st[0]

In [ ]:
#decimate to 50 sps, no need for higher frequencies
if (tr.stats.sampling_rate == 100.0):
    tr.decimate(factor=2, strict_length=False)

In [ ]:
#set start,end times for the day, from input date-time
year = int(date.split('-')[0])
month = int(date.split('-')[1])
day = int(date.split('-')[2])
start = UTCDateTime(year, month, day, 0, 0, 0)
end = start + 86400

In [ ]:
#empty 2d array for spectral amplitudes, required size, filled with zeros
nwin = 24*(3600/180) #number windows in 24 hours
z = np.zeros((249,nwin))

In [ ]:
t = start
#loop through data in 'win' sec blocks

col = 0 #columns of 2d array
while t < end:
    #slice win seconds of data
    tr10 = tr.slice(t, t + win)
    tr10.detrend(type='demean')
    npts, nyq = tr10.stats.npts, tr10.stats.sampling_rate/2.0
    #cosine taper
    tap = cosine_taper(npts,0.1)
    #calculate fft
    data = tr10.data * 1e9 #convert from m/s to nm/s
    trf = rfft(data*tap)
    a = abs(trf[1:])
    #calculate frequencies for fft
    f = np.linspace(0,nyq,npts/2)
    #interpolate to 0.1 Hz (lower resolution)
    df = np.arange(0.1,nyq,0.1) #0.1 Hz frequencies
    da = np.interp(df, f, a)
    #fill column with spectral data
    z[:,col] = da
    
    #next window
    col += 1
    t += win

In [ ]:
#array for pcolormesh, x-axis
hour = np.arange(0, 24, float(win)/float(3600))

In [ ]:
#boundaries of cmap, nm/s
vmin = 10
vmax = 5e6

In [ ]:
fig,ax = plt.subplots(figsize=(24,10))
ax.pcolormesh(hour, df, z, cmap='seismic', norm=matplotlib.colors.LogNorm(vmin=vmin, vmax=vmax))

ax.set_ylim(df.min(), df.max())
ax.set_xlim(hour.min(), hour.max())
ax.set_xticks(np.arange(hour.min(), hour.max(), 1))
ax.set_ylabel('Frequency (Hz)', fontsize=12)
ax.set_xlabel('Time (hours)', fontsize=12)
ax.grid(b=True, which='major', color='gray', linestyle='--')#, alpha=0.5)

suptitle = sta+'.'+loc+'-'+cmp+'.'+net+', '+date
plt.suptitle(suptitle, fontsize=20)
title = 'Time resolution: '+str(win)+' sec, Frequency resolution:'+str(fint)+' Hz, Colour map: LogNorm'
plt.title(title, fontsize=12)
#plt.savefig('temaari.png', dpi=200)